In [1]:
import numpy as np
from sklearn.model_selection import KFold
import os
import pandas as pd
from tqdm import tqdm
import torch

In [2]:
from video_loading import balance_videos_list
from extract_keypoints import KeypointInterpolator

folder_path = r'10 class 28 actor (different size)'
num_labels = balance_videos_list(folder_path)

interpolator = KeypointInterpolator(num_labels, frames=80)
interpolator.run()

a = np.load(f'vsl{num_labels}_data_preprocess.npy')
b = np.load(f'vsl{num_labels}_label_preprocess.npy')

print(a.shape)
print(b.shape)

Video names have been written to videos_list.csv
Minimum label: 20
Labels have been updated and saved.
{'6', '5', '4', '2', '9', '3', '0', '7', '8', '1'}
Detected 28 actors: ['01', '02', '03', '04', '05', '06', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '26', '27', '28', '29', '30', '31']

Balanced file written to: videos_list_balanced.csv

Count per label (ascending):
   3 : 28
   4 : 28
   5 : 28
   6 : 28
   7 : 28
   8 : 28
   9 : 28
   0 : 28
   1 : 28
   2 : 28


Packing: 100%|██████████| 280/280 [00:03<00:00, 84.73it/s]

(280, 2, 80, 46, 1)
(280,)


In [3]:
def k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds, destination_folder="numpy_files"):
    os.makedirs(destination_folder, exist_ok=True)

    actors = train_data['actor'].unique()
    print(f"Number of actors: {len(actors)}")
    print('-----------------------------------------------------')

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    actor_to_indices = {actor: train_data.index[train_data['actor'] == actor].tolist() for actor in actors}
    folds = [[] for _ in range(k_folds)]

    for fold, (train_actors, test_actors) in enumerate(kf.split(actors)):
        train_actors = actors[train_actors]
        test_actors = actors[test_actors]

        for actor in test_actors:
            folds[fold].extend(actor_to_indices[actor])

        tqdm.write(f"Fold {fold+1}: {len(folds[fold])} test samples")

    # Iterate over each fold to create train-test splits
    for fold in range(k_folds):
        test_indices = folds[fold]
        train_indices = [idx for f in range(k_folds) if f != fold for idx in folds[f]]

        X_train, X_test = keypoint_data[train_indices], keypoint_data[test_indices]
        y_train = np.array(label_data[train_indices], dtype=np.int64)
        y_test = np.array(label_data[test_indices], dtype=np.int64)

        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_train.npy'), X_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_train.npy'), y_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_test.npy'), X_test)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_test.npy'), y_test)

        tqdm.write(f"Processed and saved vsl{num_labels} fold {fold+1} successfully.")

if __name__ == "__main__":
    input_file_path = f"vsl{num_labels}_interpolated_keypoints.csv"
    train_data = pd.read_csv(input_file_path)

    keypoint_data = np.load(f'vsl{num_labels}_data_preprocess.npy')
    label_data = np.load(f'vsl{num_labels}_label_preprocess.npy')

    num_labels = len(np.unique(label_data))

    k_folds = 20
    k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds)


Number of actors: 28
-----------------------------------------------------
Fold 1: 20 test samples
Fold 2: 20 test samples
Fold 3: 20 test samples
Fold 4: 20 test samples
Fold 5: 20 test samples
Fold 6: 20 test samples
Fold 7: 20 test samples
Fold 8: 20 test samples
Fold 9: 10 test samples
Fold 10: 10 test samples
Fold 11: 10 test samples
Fold 12: 10 test samples
Fold 13: 10 test samples
Fold 14: 10 test samples
Fold 15: 10 test samples
Fold 16: 10 test samples
Fold 17: 10 test samples
Fold 18: 10 test samples
Fold 19: 10 test samples
Fold 20: 10 test samples
Processed and saved vsl10 fold 1 successfully.
Processed and saved vsl10 fold 2 successfully.
Processed and saved vsl10 fold 3 successfully.
Processed and saved vsl10 fold 4 successfully.
Processed and saved vsl10 fold 5 successfully.
Processed and saved vsl10 fold 6 successfully.
Processed and saved vsl10 fold 7 successfully.
Processed and saved vsl10 fold 8 successfully.
Processed and saved vsl10 fold 9 successfully.
Processed a

In [ ]:
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from feeder import FeederINCLUDE
from aagcn import Model
from augumentation import Rotate, Compose
from pytorch_lightning.utilities.migration import pl_legacy_patch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if __name__ == '__main__':
    k_folds = 20
    config = {'batch_size': 64, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    best_accuracy = 0.0
    best_fold = -1

    for fold in range(k_folds):
        print(f"Starting fold {fold + 1}/{k_folds}")
        train_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_train.npy')
        train_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_train.npy')
        val_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_test.npy')
        val_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_test.npy')

        transforms = Compose([
            Rotate(15, 80, 25, (0.5, 0.5))
        ])

        train_dataset = FeederINCLUDE(
            data_path=train_data_path,
            label_path=train_label_path,
            transform=transforms
        )
        val_dataset = FeederINCLUDE(
            data_path=val_data_path,
            label_path=val_label_path
        )

        train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        model = Model(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                      graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                      learning_rate=config['learning_rate'], weight_decay=config['weight_decay'])

        # Path pre-trained checkpoint file
        checkpoint_path = "autsl_vsl199-aagcn-fold=7-v1.ckpt"

        with pl_legacy_patch():
            checkpoint = torch.load(checkpoint_path, map_location=device)

        state_dict = checkpoint['state_dict']
        filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}
        model.load_state_dict(filtered_state_dict, strict=False)

        callbacks = [
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="valid_accuracy",
                mode="max",
                every_n_epochs=1,
                filename=f'autsl_vsl{num_labels}-aagcn-fold={fold+1}'
            ),
        ]

        trainer = pl.Trainer(max_epochs=10, accelerator="auto", check_val_every_n_epoch=1,
                             devices=1, callbacks=callbacks)

        trainer.fit(model, train_dataloader, val_dataloader)
        val_accuracy = trainer.callback_metrics['valid_accuracy'].item()
        print(f"Fold {fold + 1} finished with validation accuracy: {val_accuracy:.4f}")

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_fold = fold + 1

    print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")


c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting fold 1/20
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (41, 37), (37, 39), (39, 38), (38, 40), (41, 22), (22, 23), (23, 21), (21, 24), (22, 26), (26, 27), (27, 25), (25, 28), (26, 34), (34, 35), (35, 33), (33, 36), (41, 30), (30, 31), (31, 29), (29, 32), (20, 16), (16, 18), (18, 17), (17, 19), (20, 1), (1, 2), (2, 0), (0, 3), (1, 5), (5, 6), (6, 4), (4, 7), (5, 13), (13, 14), (14, 12), (12, 15), (20, 9), (9, 10), (10, 8), (8, 11), (42, 45), (45, 20), (42, 43), (43, 44), (44, 41)]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name    | Type               | Params | Mode 
--------------------------------------------------------
0  | data_bn | BatchNorm1d        | 184    | train
1  | l1      | TCN_GCN_unit       | 52.4 K | train
2  | l2      | TCN_GCN_unit       | 69.9 K | train
3  | l3      | TCN_GCN_unit       | 69.9 K | train
4  | l4      | TCN_GCN_unit       | 69.9 K | train
5  | l5      | TCN_GCN_unit       | 232 K  | train
6  | l6      | TCN_GCN_unit       | 252 K  | train
7  | l7      | TCN_GCN_unit       | 252 K  | train
8  | l8      | TCN_GCN_unit       | 892 K  | train
9  | l9      | TCN_GCN_unit       | 973 K  | train
10 | l10     | TCN_GCN_unit       | 973 K  | train
11 | fc      | Linear             | 2.6 K  | train
12 | loss    | CrossEntropyLoss   | 0      | train
13 | metric  | MulticlassAccuracy | 0      | train
--------------------------------------------------------
3.8 M    

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Epoch 9: 100%|██████████| 5/5 [00:57<00:00,  0.09it/s, v_num=6, train_accuracy_step=0.600, train_loss_step=1.010, valid_accuracy=0.750, valid_loss=0.836, train_accuracy_epoch=0.954, train_loss_epoch=0.165]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:57<00:00,  0.09it/s, v_num=6, train_accuracy_step=0.600, train_loss_step=1.010, valid_accuracy=0.750, valid_loss=0.836, train_accuracy_epoch=0.954, train_loss_epoch=0.165]
Fold 7 finished with validation accuracy: 0.7500
Starting fold 8/20
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (41, 37), (37, 39), (39, 38), (38, 40), (41, 22), (22, 23), (23, 21), (21, 24), (22, 26), (26, 27), (27, 25), (25, 28), (26, 34), (34, 35), (35, 33), (33, 36), (41, 30), (30, 31), (31, 29), (29, 32), (20, 16), (16, 18), (18, 17), (17, 19), (20, 1), (1, 2), (2, 0), (0, 3), (1, 5), (5, 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name    | Type               | Params | Mode 
--------------------------------------------------------
0  | data_bn | BatchNorm1d        | 184    | train
1  | l1      | TCN_GCN_unit       | 52.4 K | train
2  | l2      | TCN_GCN_unit       | 69.9 K | train
3  | l3      | TCN_GCN_unit       | 69.9 K | train
4  | l4      | TCN_GCN_unit       | 69.9 K | train
5  | l5      | TCN_GCN_unit       | 232 K  | train
6  | l6      | TCN_GCN_unit       | 252 K  | train
7  | l7      | TCN_GCN_unit       | 252 K  | train
8  | l8      | TCN_GCN_unit       | 892 K  | train
9  | l9      | TCN_GCN_unit       | 973 K  | train
10 | l10     | TCN_GCN_unit       | 973 K  | train
11 | fc      | Linear             | 2.6 K  | train
12 | loss    | CrossEntropyLoss   | 0      | train
13 | metric  | MulticlassAccuracy | 0      | train
--------------------------------------------------------
3.8 M    

Epoch 1:  60%|██████    | 3/5 [00:41<00:27,  0.07it/s, v_num=7, train_accuracy_step=0.868, train_loss_step=0.690, valid_accuracy=0.150, valid_loss=38.20, train_accuracy_epoch=0.414, train_loss_epoch=6.020]

In [ ]:
print(f"The highest validation accuracy achieved of autsl vsl{num_labels} is {best_accuracy:.4f} from fold {best_fold}.")